In [1]:
import os
from ultralytics import YOLO
import torch
import cv2
from tqdm import tqdm

In [2]:
data_dir = "D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\splits"  # Папка, содержащая подпапки fold_1, fold_2, fold_3, fold_4
output_dir = "D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\res" # Папка для сохранения результатов обучения и моделей

In [3]:
def create_yolov8_yaml(train_dir, test_dir, yaml_path):
    classes = ['машина', 'бордюр', 'дверь', 'стена', 'лестница', 'спуск', 'яма', 'люк', 'ограждение', 'трамвайные пути']
    num_classes = len(classes)

    yaml_content = f"""
    train: {os.path.join(train_dir, "images")}
    val: {os.path.join(test_dir, "images")}

    nc: {num_classes}
    names: {classes}
    """
    with open(yaml_path, "w") as f:
        f.write(yaml_content)
    print(f"создан YAML: {yaml_path}")

In [ ]:
k_folds = 4
for fold in range(8, 9):
    fold_dir = os.path.join(data_dir, f"fold_{fold}")
    train_dir = os.path.join(fold_dir, "train")
    test_dir = os.path.join(fold_dir, "test")

    map50_95_values = []
    losses = []

    save_dir = output_dir

    yaml_path = os.path.join(fold_dir, "data.yaml")
    if not os.path.exists(yaml_path):
        create_yolov8_yaml(train_dir, test_dir, yaml_path)

    print(f"выполняется тренировка для папки номер {fold} из {k_folds}")

    if fold == 0:
        weights_path = 'yolov8n.pt'
    else:
        weights_path = os.path.join(save_dir, f"yolov8n_fold_{fold - 1}", "weights", "best.pt")

    model = YOLO(weights_path)

    results = model.train(
        data=yaml_path,
        epochs=70,
        imgsz=640,
        device="cpu",
        project = save_dir,
        name=f"yolov8n_fold_{fold}",
        save=True,
        optimizer='AdamW',
        patience=10,
        lr0=0.01,
        weight_decay=0.0005,
        batch = 32)


    best_weights_path = os.path.join(save_dir, f"yolov8n_fold_{fold}", "weights", "best.pt")
    print(f"результаты обработки папки номер {fold}...")
    
    folder_path = f"D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\res\\metrics_{fold}"
    filename = f'metrics.txt'
    filepath = os.path.join(folder_path, filename)

    #if os.path.exists(filepath):
    #    os.remove(filepath)
    metrics = model.val(data=f"D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\splits\\fold_{fold}\\data.yaml")
    try:
    
        file_descriptor = os.open(filepath, os.O_WRONLY | os.O_CREAT)

        metrics_string = f"  Precision: {metrics.results_dict['metrics/precision(B)']}\n" \
                     f"  Recall: {metrics.results_dict['metrics/recall(B)']}\n" \
                     f"  mAP@0.5: {metrics.results_dict['metrics/mAP50(B)']}\n" \
                     f"  mAP@0.5:0.95: {metrics.results_dict['metrics/mAP50-95(B)']}\n"
        try:
            file_descriptor = os.open(filepath, os.O_WRONLY | os.O_CREAT)
            print("файл успешно открыт.")
    
            os.write(file_descriptor, metrics_string.encode())
            print("метрики записаны.")
        except OSError as e:
            print(f"ошибка: {e}")
        except Exception as e:
            print(f"ошибка: {e}")

        os.write(file_descriptor, metrics_string.encode())


        for i, class_name in enumerate(model.names):
            class_string = f"\nClass: {class_name}\n" \
                           f"  Precision: {metrics.box.p[i]}\n" \
                           f"  Recall: {metrics.box.r[i]}\n" 
            os.write(file_descriptor, class_string.encode())
    
        os.close(file_descriptor)
    
        print(f"метрики сохранены как {filename}")

    except OSError as e:
        print(f"ошибка записи в файле: {e}")